In [8]:
from flask import Flask, request, jsonify
from PIL import Image, ImageOps
import ultralytics
ultralytics.checks()
import re
from ultralytics import YOLO
import cv2
import rembg
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
import easyocr
from matplotlib.patches import Rectangle
import numpy as np
import os
import shutil
from flask import Flask, request, jsonify
import matplotlib.pyplot as plt

Ultralytics YOLOv8.0.145 🚀 Python-3.7.5 torch-1.13.1+cu117 CPU (Intel Core(TM) i7-10750H 2.60GHz)
Setup complete ✅ (12 CPUs, 15.4 GB RAM, 128.0/233.7 GB disk)


In [15]:
def Arabic_OCR(img):

        img_fr_or = img



        ################# INPUT ##################

        model = YOLO("version 2 best.pt")
        Fr_w = img_fr_or.copy()

        ##########################################


        ############## REMOVE BACK GROUND ###############

        def remove_background(img):
            # Convert the input image to bytes
            img = np.array(img)
            s = img.copy()
            # Step 3: Convert to RGB and swap red and blue channels manually
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Step 4: Convert back to PIL Image
            img = Image.fromarray(img)
            buffered = BytesIO()
            img.save(buffered, format="PNG")
            encoded_image = buffered.getvalue()

            # Remove the background
            output_bytes = rembg.remove(encoded_image)

            # Convert the output bytes back to an image
            output_image = Image.open(BytesIO(output_bytes))

            return output_image



        ############## Perspective RATIO MODIFYING ###############


        def rectify_id_card(img):
            # Remove the background
            img = np.array(remove_background(img))

            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Change from BGR to RGB

            # Find contours
            contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Assuming the largest contour is the ID card
            card_contour = max(contours, key=cv2.contourArea)

            # Find corners of the card
            epsilon = 0.05 * cv2.arcLength(card_contour, True)
            approx = cv2.approxPolyDP(card_contour, epsilon, True)

            # Reorder the points to ensure they are in clockwise order
            corners = np.array(approx).reshape(-1, 2)
            ordered_corners = np.zeros_like(corners)

            # Calculate the centroid of the points
            centroid = np.mean(corners, axis=0)

            # Sort the points based on their angle from the centroid
            angles = np.arctan2(corners[:, 1] - centroid[1], corners[:, 0] - centroid[0])
            sorted_indices = np.argsort(angles)

            # Reorder the corners
            for i in range(4):
                ordered_corners[i] = corners[sorted_indices[i]]

            # Define the coordinates of the corners of the desired rectangle
            width, height = 840, 530
            dst_corners = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype='float32')

            # Calculate the perspective transform matrix
            M = cv2.getPerspectiveTransform(ordered_corners.astype('float32'), dst_corners)

            # Apply the perspective transformation
            result = cv2.warpPerspective(img, M, (width, height))
            result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
            return result
        #####################################################

        Fr_w = rectify_id_card(img_fr_or.copy())


        # Initialize easyocr for Arabic and English
        reader = easyocr.Reader(['fa', 'ar'], verbose=True)  # 'fa' is the code for Persian, which shares the script with Arabic.

        def yolo_ID(img):
            kernel = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])

            # Apply the convolution operation
            img = cv2.filter2D(img, -1, kernel)
            inverted_image = cv2.bitwise_not(img)
            img = inverted_image
            predict = model.predict(img, save=True, save_txt=True, conf=0.7)
            outs = open("/home/mo/ocr_app/runs/detect/predict/labels/image0.txt" , "r").readlines()
            label = ''
            for i in outs:
              label = label+i

            classes = ['0','1','2','3','4','5','6','7','8','9']
            shutil.rmtree("/home/mo/ocr_app/runs")
            # Extract and sort classes based on x-coordinate
            detected_entries = []
            for line in label.strip().split("\n"):
                class_id, x_coord, _, _, _ = map(float, line.split())
                detected_entries.append((x_coord, classes[int(class_id)]))

            detected_entries.sort(key=lambda x: x[0])  # Sort by x-coordinate

            # Extract sorted detected classes from entries
            detected_classes_sorted = [entry[1] for entry in detected_entries]

            # Print detected classes
            return ''.join(detected_classes_sorted)


        def extract_arabic_english(text):
            pattern = r'[a-zA-Z\u0600-\u06FF\s]+'
            matches = re.findall(pattern, text)
            cleaned_text = ' '.join(matches)
            return cleaned_text

        def ocr_image(mask, reader):
            results = reader.readtext(mask,text_threshold=0.7, low_text=0.3,paragraph = True)  # Set the allowlist to Arabic script
            detected_texts = [item[1] for item in results]
            full_text = ' '.join(detected_texts)
            cleaned_text = extract_arabic_english(full_text)
            return cleaned_text, results


        def process_image(image, image_name='Image.jpg'):
            # Define a sharpening kernel

            img = image.copy()
            img = cv2.resize(img, (840, 530))
            alpha = 1
            beta = 1
            img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
            masks = {
                "name_mask": img[140:240, 430:],
                "add_mask": img[240:380, 450:],
                "Id_mask": img[350:, 350:]
            }
            texted = []
            # Reverse the order of processing masks
            for mask_name, mask in (list(masks.items())):
                cleaned_text, results = ocr_image(mask, reader)

                cleaned_text = cleaned_text.split()
                for tx in range(len(cleaned_text)):
                  if cleaned_text[tx][:3] == 'محم':
                    cleaned_text[tx] = 'محمد'

                texted.append(cleaned_text)
            return texted


        # Process the first image
        Name, Location, _= process_image(Fr_w)
        nName = ' '
        for n in Name:
          nName = nName +' '+ n
        nLocation = ' '
        for n in Location:
          nLocation = nLocation +' '+ n
        ID_N = yolo_ID(Fr_w)
        #print(nName, nLocation, ID_N, Job, Gender, Religion)

        return ID_N

In [16]:
app = Flask(__name__)

@app.route("/")

@app.route('/image', methods=['POST'])
def post_example():
    # Access request data using request object
    data = request.json  # Assumes JSON data in the request body

    # Return the received data in the response
    return jsonify(data)

@app.route('/upload_photo', methods=['POST'])
def upload_photo():
    if 'photo' not in request.files:
        return jsonify({'message': 'No file part'}), 400

    photo = request.files['photo']
    if photo.filename == '':
        return jsonify({'message': 'No selected file'}), 400

    # You can process the file without saving it
    # For example, print the file name and content type
    print('Received File:', photo.filename)
    print('Content Type:', photo.content_type)

    return jsonify({'message': 'File received'})

# paste here the code of ur machine learning model 


img = cv2.imread('test2.jpg')
ID_N = Arabic_OCR(img)


# print the result of the model
print(ID_N)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.

0: 416x640 2 0s, 3 1s, 3 2s, 1 4, 1 6, 2 7s, 1 9, 11356.2ms
Speed: 51.6ms preprocess, 11356.2ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)
Results saved to /home/mo/ocr_app/runs/detect/predict
1 label saved to /home/mo/ocr_app/runs/detect/predict/labels


2921206140717
